# Texts and Documents

---






## Introduction

This section documents the model for documents that contain text, including artworks such as medieval manuscripts, archival material such as letters, ledgers or diaries, scholarly communication such as journals, articles and monographs, digital objects such as web pages, or any other sort of written communication.

The intent is not to be an all-encompassing model that would be suitable for a graph based library management system, archival finding aids, or catalog of any digital resource, but instead to provide sufficient description that the object is identified, understandable and able to be referenced within other more specific systems and ontologies.  This model is intended to be enough to use for the basic use cases of referencing texts that are related to artworks, regardless of whether they are held in a library, archive, museum, or on the internet.

Notably, it does not attempt to reproduce the formalisms of [FRBR](https://www.ifla.org/node/2016) and similar abstract models, as manifested by [FRBRoo](https://www.ifla.org/publications/node/11240), [BibFrame](http://www.loc.gov/bibframe/docs/index.html), or other complex conceptual hierarchies, but instead provide as simple as possible a model to accomplish core bibliographic reference tasks. 

## Physical Objects, Conceptual Texts

The first distinction that is needed is between the physical or digital carrier of the text, and the text itself.  Like the `VisualItem` pattern for the [artwork's visual content shown by objects](/model/object/aboutness/#depiction), the `LinguisticObject` that represents the text of a work can be `carried` by more than one `HumanMadeObject` and `digitally_carried_by` `DigitalObject` instances.  In this way, all of the copies of a particular book carry the same information content, that only needs to be described once and can act as a single connection point.

Like other `LinguisticObject`s, it can have a `content` property for the actual text of the work, classifications, languages and so forth.

!!! note "Oral Histories"
    The linguistic content does not need to be ever written down with symbols for it to be considered a `LinguisticObject`, it could be transmitted entirely through oral traditions. Equally, an interview could be recorded without ever being transcribed and the information transmitted would still be a `LinguisticObject` as it is transmitted through the use of human language, rather than visual depictions. 


__Example:__

Yale University Library holds a copy of Koot's book about The Night Watch.



In [1]:
# Step 1: Import cromulent library
from cromulent import model, vocab

# Step 2: Configure factory settings
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# Step 3: Create the main object (Yale's copy of Koot's book)
# Who: Ton Koot (author), Meulenhoff International (publisher), Yale University Library (holder)
# What: Book carrying linguistic content about The Night Watch
# Why: To document a physical book that carries textual content
book = vocab.Book(
    ident="yul_10801219/1",
    label="Yale's copy of Koot's Night Watch"
)

# Step 4: Create related objects and relationships
# What identifies this book: Primary name and system number
book.identified_by = vocab.PrimaryName(
    content="Rembrandt's Night Watch. A Fascinating Story"
)
book.identified_by = vocab.SystemNumber(content="mfhd:10801219")

# What physical description: 92 pages with illustrations
book.referred_to_by = vocab.PhysicalStatement(
    content="92 p. with illus."
)

# What linguistic content it carries: Text about The Night Watch
text_content = model.LinguisticObject(
    ident="koot_nightwatch",
    label="Content of Koot's Night Watch"
)
book.carries = text_content

# Step 5: Display the generated JSON-LD
print(model.factory.toString(book, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "http://lod.example.org/museum/HumanMadeObject/yul_10801219/1",
  "type": "HumanMadeObject",
  "_label": "Yale's copy of Koot's Night Watch",
  "classified_as": [
    {
      "id": "http://vocab.getty.edu/aat/300028051",
      "type": "Type",
      "_label": "Book",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300435443",
          "type": "Type",
          "_label": "Type of Work"
        }
      ]
    }
  ],
  "identified_by": [
    {
      "type": "Name",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300404670",
          "type": "Type",
          "_label": "Primary Name"
        }
      ],
      "content": "Rembrandt's Night Watch. A Fascinating Story"
    },
    {
      "type": "Identifier",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300435704",
          "type": "Type",
          "_label": "System-Assigned Numbe



## Common Features

The common features of the model are also applicable to the `LinguisticObject`s used to represent texts.  They must have an `id` and `type`, they should have a `label`, they should have a `classified_as` relationship to a `Type` that further describes the sort of object, and so forth. They may also have a `language` property, that references the `Language` of the text.

The title of a text is captured using the `Name` pattern, and assigned identifiers using the `Identifier` pattern, both with the `identified_by` relationship.

__Example:__

The textual content of Koot's work.



In [2]:
# Step 1: Import cromulent library
from cromulent import model, vocab

# Step 2: Configure factory settings
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# Step 3: Create the main object (Textual content of Koot's book)
# Who: Ton Koot (author)
# What: MonographText representing the book's content
# Why: To document the abstract linguistic content of the book
text_content = vocab.MonographText(
    ident="koot_nightwatch/1",
    label="Content of Koot's Night Watch"
)

# Step 4: Create related objects and relationships
# What identifies this text: Primary name and system number
text_content.identified_by = vocab.PrimaryName(
    content="Rembrandt's Night Watch. A Fascinating Story"
)
text_content.identified_by = vocab.SystemNumber(content="75441784")

# What language: English
text_content.language = vocab.instances['english']

# Step 5: Display the generated JSON-LD
print(model.factory.toString(text_content, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "http://lod.example.org/museum/LinguisticObject/koot_nightwatch/1",
  "type": "LinguisticObject",
  "_label": "Content of Koot's Night Watch",
  "classified_as": [
    {
      "id": "http://vocab.getty.edu/aat/300060417",
      "type": "Type",
      "_label": "Monograph",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300435443",
          "type": "Type",
          "_label": "Type of Work"
        }
      ]
    }
  ],
  "identified_by": [
    {
      "type": "Name",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300404670",
          "type": "Type",
          "_label": "Primary Name"
        }
      ],
      "content": "Rembrandt's Night Watch. A Fascinating Story"
    },
    {
      "type": "Identifier",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300435704",
          "type": "Type",
          "_label": "System-Assigned



## Creation and Publication

The production of the physical carriers of texts uses the same model as for other physical objects, and may be of interest for manuscripts, very early printed works (incunabula), letters or other similar documents, however the factory details for a specific modern book are likely of much less importance to capture.

There are two primary text-specific activities that are captured -- the creation of the text and its publication.  The text is created by a `Creation` activity of the author, but then there is a publishing `Activity` (_aat:300054686_) carried out by the publishing organization, for the same entity. 

__Example:__

The authorship and publication of the text.



In [3]:
# Step 1: Import cromulent library
from cromulent import model, vocab

# Step 2: Configure factory settings
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# Step 3: Create the main object (Textual content of Koot's book)
# Who: Ton Koot (author), Meulenhoff International (publisher)
# What: MonographText with authorship and publication information
# Why: To document the creation and publication of the text
text_content = vocab.MonographText(
    ident="koot_nightwatch/2",
    label="Content of Koot's Night Watch"
)

# Step 4: Create related objects and relationships
# Creation event: Author wrote the text
creation = model.Creation(label="Koot's writing of the work")
creation.carried_out_by = model.Person(
    ident="koot",
    label="Ton Koot"
)
text_content.created_by = creation

# Publishing activity: Publisher published the text
publishing = vocab.Publishing(label="MI's Publishing")
publishing.carried_out_by = model.Group(
    ident="meulenhoff",
    label="Meulenhoff International"
)

# When: 1969
timespan = model.TimeSpan()
timespan.begin_of_the_begin = "1969-01-01T00:00:00Z"
timespan.end_of_the_end = "1969-12-31T23:59:59Z"
publishing.timespan = timespan

text_content.used_for = publishing

# Step 5: Display the generated JSON-LD
print(model.factory.toString(text_content, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "http://lod.example.org/museum/LinguisticObject/koot_nightwatch/2",
  "type": "LinguisticObject",
  "_label": "Content of Koot's Night Watch",
  "classified_as": [
    {
      "id": "http://vocab.getty.edu/aat/300060417",
      "type": "Type",
      "_label": "Monograph",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300435443",
          "type": "Type",
          "_label": "Type of Work"
        }
      ]
    }
  ],
  "created_by": {
    "type": "Creation",
    "_label": "Koot's writing of the work",
    "carried_out_by": [
      {
        "id": "http://lod.example.org/museum/Person/koot",
        "type": "Person",
        "_label": "Ton Koot"
      }
    ]
  },
  "used_for": [
    {
      "type": "Activity",
      "_label": "MI's Publishing",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300054686",
          "type": "Type",
          "_label": "Publis



## Structure

The textual structure can be modeled with the partitioning of `LinguisticObject`s via the `part_of` property, in the same way that the parts of a physical object can be partitioned.  Thus the content of an Article can be part of the containing Issue, which is part of a Volume, which is part of the Journal or other periodical.  Similarly Chapters can be part of a Book or Proceedings, particular entries within a catalog, and so forth. 

Note that unless it is important to create a separate record (for example the chapter is written by a different author, or there are other significant details), then it is also possible to just add the description of the structure as a note, using the Statement pattern.

__Example:__



In [4]:
# Step 1: Import cromulent library
from cromulent import model, vocab

# Step 2: Configure factory settings
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# Step 3: Create the main object (Chapter 1 of Koot's book)
# Who: Ton Koot (author)
# What: ChapterText representing part of the book
# Why: To document the hierarchical structure of the text
chapter = vocab.ChapterText(
    ident="koot_nightwatch_ch1/1",
    label="Chapter 1 of Koot"
)

# Step 4: Create related objects and relationships
# What identifies this chapter: Title
chapter.identified_by = vocab.PrimaryName(content="Introduction")

# What larger text it is part of: The full book
chapter.part_of = model.LinguisticObject(
    ident="koot_nightwatch",
    label="Koot's Night Watch"
)

# Step 5: Display the generated JSON-LD
print(model.factory.toString(chapter, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "http://lod.example.org/museum/LinguisticObject/koot_nightwatch_ch1/1",
  "type": "LinguisticObject",
  "_label": "Chapter 1 of Koot",
  "classified_as": [
    {
      "id": "http://vocab.getty.edu/aat/300311699",
      "type": "Type",
      "_label": "Chapter",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300435443",
          "type": "Type",
          "_label": "Type of Work"
        }
      ]
    }
  ],
  "identified_by": [
    {
      "type": "Name",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300404670",
          "type": "Type",
          "_label": "Primary Name"
        }
      ],
      "content": "Introduction"
    }
  ],
  "part_of": [
    {
      "id": "http://lod.example.org/museum/LinguisticObject/koot_nightwatch",
      "type": "LinguisticObject",
      "_label": "Koot's Night Watch"
    }
  ]
}




## Pages

Textual content is typically presented on pages or folios. As there might be many physical copies with the same structure, it is common to describe the pagination of the content as it applies to the content in general, rather than the many physical objects that carry that content. This is justifiable, mostly for convenience, but also because the text is divided up into sections which happen to correspond to the physical carrier's structure. Pagination (_aat:300200294_) or Foliation (_aat:300200662_) statements are the most common way to represent this, as simple descriptive fields following the core statement pattern. 

The structured way is to have a Dimension associated with the object or work that gives the count of pages. This follows the regular pattern for counting parts. The easiest way to describe the page range of a particular section of a larger collection is with a pagination statement.  The disadvantage of the approach is that it is not computationally available for processing.

__Example:__

The introduction chapter to Koot's book is 10 pages long.



In [5]:
# Step 1: Import cromulent library
from cromulent import model, vocab

# Step 2: Configure factory settings
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# Step 3: Create the main object (Chapter 1 of Koot's book)
# Who: Ton Koot (author)
# What: ChapterText with pagination information
# Why: To document the page count and range for a chapter
chapter = vocab.ChapterText(
    ident="koot_nightwatch_ch1/2",
    label="Chapter 1 of Koot"
)

# Step 4: Create related objects and relationships
# What identifies this chapter: Title
chapter.identified_by = vocab.PrimaryName(content="Introduction")

# What pagination: Page range "5 - 15"
chapter.referred_to_by = vocab.PaginationStatement(content="5 - 15")

# What page count: 10 pages
dimension = model.Dimension(label="10 pages")
dimension.value = 10
dimension.classified_as = model.Type(
    ident="http://vocab.getty.edu/aat/300404433",
    label="Count Of"
)
dimension.unit = model.MeasurementUnit(
    ident="http://vocab.getty.edu/aat/300194222",
    label="Pages"
)
chapter.dimension = dimension

# Step 5: Display the generated JSON-LD
print(model.factory.toString(chapter, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "http://lod.example.org/museum/LinguisticObject/koot_nightwatch_ch1/2",
  "type": "LinguisticObject",
  "_label": "Chapter 1 of Koot",
  "classified_as": [
    {
      "id": "http://vocab.getty.edu/aat/300311699",
      "type": "Type",
      "_label": "Chapter",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300435443",
          "type": "Type",
          "_label": "Type of Work"
        }
      ]
    }
  ],
  "identified_by": [
    {
      "type": "Name",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300404670",
          "type": "Type",
          "_label": "Primary Name"
        }
      ],
      "content": "Introduction"
    }
  ],
  "referred_to_by": [
    {
      "type": "LinguisticObject",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300435440",
          "type": "Type",
          "_label": "Pagination Statement",
  



## References to other Entities

Textual content can be `about` any of the other entities described using the model. These should be the core entities rather than structural entities like a `Dimension` or `Identifier`, and this is required to implement the API.

__Example:__

Koot's book is about The Night Watch.



In [6]:
# Step 1: Import cromulent library
from cromulent import model, vocab

# Step 2: Configure factory settings
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# Step 3: Create the main object (Textual content of Koot's book)
# Who: Ton Koot (author)
# What: MonographText about The Night Watch
# Why: To document that the text is about a specific artwork
text_content = vocab.MonographText(
    ident="koot_nightwatch/3",
    label="Content of Koot's Night Watch"
)

# Step 4: Create related objects and relationships
# What the text is about: The Night Watch painting
text_content.about = model.HumanMadeObject(
    ident="nightwatch",
    label="The Night Watch"
)

# Step 5: Display the generated JSON-LD
print(model.factory.toString(text_content, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "http://lod.example.org/museum/LinguisticObject/koot_nightwatch/3",
  "type": "LinguisticObject",
  "_label": "Content of Koot's Night Watch",
  "classified_as": [
    {
      "id": "http://vocab.getty.edu/aat/300060417",
      "type": "Type",
      "_label": "Monograph",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300435443",
          "type": "Type",
          "_label": "Type of Work"
        }
      ]
    }
  ],
  "about": [
    {
      "id": "http://lod.example.org/museum/HumanMadeObject/nightwatch",
      "type": "HumanMadeObject",
      "_label": "The Night Watch"
    }
  ]
}




## Abstract Work

It is sometimes valuable to relate the content with a higher level abstraction which does not even distinguish between text, audio, video, image or other forms, let alone the various editions, translations or other manifestations of the idea. Some general examples would include The Lord of the Rings, as conceptualized and originally written by Tolkien in English, but subsequently turned into a films, such as by Peter Jackson, and other interpretations of the same abstract work. While not a strict LRM or FRBR model, this does allow the mapping of the "Work" level into an appropriately abstract class in Linked Art.

The abstract work has all of the same features as the Linguistic Object with the exception of `language`. Linguistic, Visual and other Abstract works can be `conceptually_part_of` Abstract works to connect them together in a hierarchy. 

__Example:__

The text of Koot's book is conceptually part of a larger more abstract work of his conceptualization of the research and discussion, which might have multiple editions, translations or instantiations.



In [7]:
# Step 1: Import cromulent library
from cromulent import model, vocab

# Step 2: Configure factory settings
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# Step 3: Create the main object (Textual content of Koot's book)
# Who: Ton Koot (author)
# What: LinguisticObject conceptually part of an abstract work
# Why: To document the relationship between specific text and abstract concept
text_content = model.LinguisticObject(
    ident="koot_nightwatch/1",
    label="Content of Koot's Night Watch"
)

# Step 4: Create related objects and relationships
# What identifies this text: Title
text_content.identified_by = vocab.PrimaryName(
    content="Rembrant's Night Watch. A Fascinating Story"
)

# What abstract work it is conceptually part of: Koot's conceptualization
text_content.conceptually_part_of = model.PropositionalObject(
    ident="koot_idea",
    label="Koot's Conceptualization"
)

# Step 5: Display the generated JSON-LD
print(model.factory.toString(text_content, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "http://lod.example.org/museum/LinguisticObject/koot_nightwatch/1",
  "type": "LinguisticObject",
  "_label": "Content of Koot's Night Watch",
  "identified_by": [
    {
      "type": "Name",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300404670",
          "type": "Type",
          "_label": "Primary Name"
        }
      ],
      "content": "Rembrant's Night Watch. A Fascinating Story"
    }
  ],
  "conceptually_part_of": [
    {
      "id": "http://lod.example.org/museum/PropositionalObject/koot_idea",
      "type": "PropositionalObject",
      "_label": "Koot's Conceptualization"
    }
  ]
}
